Affordance Dataset
==========
Parsing affordance raw data, do some statistics, and create the dataset for next steps.

The raw files, AMT\*.csv, is first processed through

    tr '\r' '\n'

and then renamed to raw[0-9].csv

Note that raw1.csv has a slightly different format from the rest of the files.

Does raw1.csv / AMTraw.csv have the correct affordance category?

In [5]:
import glob
import csv
import os
from collections import Counter

In [2]:
rawDataPathPattern = 'rawData/raw?.csv'
filePathes = glob.glob(rawDataPathPattern)
print len(filePathes), filePathes

5 ['rawData/raw2.csv', 'rawData/raw3.csv', 'rawData/raw1.csv', 'rawData/raw5.csv', 'rawData/raw4.csv']


In [46]:
imageHeader = 'Input.Image'
optionsHeaderPrefix = 'Input.Option'
answerHeader = 'Answer.Set1'
posOptionHeader = 'Option'

cnt = 0
responses = Counter()
for filePath in filePathes:
    if filePath == 'rawData/raw1.csv': continue # blocking questionable dataset
    with open(filePath) as fin:
        fReader = csv.reader(fin, skipinitialspace=True)
        header = None
        for row in fReader:
            if header == None:
                header = row
            else:
                image = None
                options = {}
                positiveResponses = set()
                if len(header) != len(row):
                    print 'ERROR', header, row
                for h, e in zip(header, row):
                    if h == imageHeader:
                        image = os.path.splitext(e[e.rfind('/')+1:])[0]
                    if h.startswith(optionsHeaderPrefix):
                        opId = int(h[len(optionsHeaderPrefix):])
                        tmp = e.lower()
                        if tmp.find('>') != -1 and tmp.rfind('<') != -1:
                            tmp = tmp[tmp.find('>')+1 : tmp.rfind('<')]
                        tmp = tmp.strip()
                        options[opId] = tmp
#                         if tmp == 'work': print e, h, filePath
                    if h == answerHeader:
                        for pos in e.split('|'):
                            if len(pos) == 0:
                                continue
                            positiveResponses.add(int(pos[len(posOptionHeader):]))
                    if image == None or len(options) == None or len(positiveResponses - set(options.keys())) != 0:
                        print 'ERROR', header, row
                    for opId, opName in options.iteritems():
                        if opId in positiveResponses:
                            responses[(image, opName, True)] += 1
                        else:
                            responses[(image, opName, False)] += 1

imageList = set()
optionList = set()
uniqueImageOption = set()
totalResponses = 0

for k, v in responses.iteritems():
    totalResponses += v
    imageList.add(k[0])
    optionList.add(k[1])
    uniqueImageOption.add(k[0:2])

imageList = sorted(list(imageList))
optionList = sorted(list(optionList))
uniqueImageOption = sorted(list(uniqueImageOption))

unanimousImageOption = 0
for image, option in uniqueImageOption:
    if (responses[image, option, True] == 0) != (responses[image, option, False] == 0):
        unanimousImageOption += 1

print len(imageList)
print len(optionList)
print len(uniqueImageOption)
print totalResponses
print unanimousImageOption

9800
228
260002
6528284
240487


In [48]:
for o in optionList:
    print o

appliance repair & maintenance (self)
architecture and engineering work
arts & crafts
arts & crafts with children
arts / design / entertainment / sports / media work
attending child's events
attending meetings for personal interest
attending movies
attending museums
attending or hosting parties
attending religious services
attending school-related meetings & conferences
attending the performing arts
banking
biking
boating
bowling
building & repairing furniture
building and grounds cleaning and maintenance work
business and financial operations work
buying/selling real estate
camping
civic obligations
cleaning home exterior
collecting as a hobby
community and social work
comparison shopping
computer and mathematical work
computer use (not games)
construction and extraction work
dancing
doing aerobics
doing gymnastics
doing martial arts
eating & drinking
education and library work
education-related administrative activities
email
exercising & playing with animals
exterior home repair & d

After we remove AMTraw.csv, there is still one more affordance category. Is the extra one "work"?

"Work" appears in both raw2 and raw4, only at option 17